In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [3]:
# Read the file
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

n = 400  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

start_node = 12
end_node = 34

In [6]:
edges.take(5)

[(12, 2241), (12, 13349), (12, 41873), (12, 82473), (12, 414853)]

In [10]:
n_verts = edges.count()

In [15]:
unvisited = edges.map(lambda x: x[0]).distinct().map(lambda x: (x, n_verts))

In [16]:
unvisited.take(10)

[(514, 54485),
 (518, 54485),
 (519, 54485),
 (521, 54485),
 (12, 54485),
 (13, 54485),
 (14, 54485),
 (15, 54485),
 (16, 54485),
 (17, 54485)]

In [76]:
def attach_node(x):
    return (x[1][1], x[1][0] + [x[1][1]])

In [101]:
paths = sc.parallelize([(start_node, [start_node, ])])
counter = 0
while True:
    
    paths = paths.join(forward_edges).map(attach_node)
    completed_paths = paths.filter(lambda x: x[0] == end_node)
    if completed_paths.count() > 0:
        print(completed_paths.collect())
        break
    
    counter += 1
    print(counter)
    #break

1
2
3
4
5
6
7


KeyboardInterrupt: 

In [93]:
paths.collect()

[(2241, [12, 2241]),
 (13349, [12, 13349]),
 (41873, [12, 41873]),
 (82473, [12, 82473]),
 (414853, [12, 414853]),
 (755452, [12, 755452]),
 (758983, [12, 758983]),
 (793023, [12, 793023]),
 (794748, [12, 794748]),
 (806280, [12, 806280]),
 (873731, [12, 873731]),
 (884271, [12, 884271]),
 (959551, [12, 959551]),
 (1016301, [12, 1016301]),
 (1070441, [12, 1070441]),
 (1190051, [12, 1190051]),
 (1190411, [12, 1190411]),
 (1467871, [12, 1467871]),
 (2039131, [12, 2039131]),
 (2049521, [12, 2049521]),
 (2683161, [12, 2683161]),
 (3038461, [12, 3038461]),
 (3140001, [12, 3140001]),
 (3171031, [12, 3171031]),
 (3259651, [12, 3259651]),
 (3492361, [12, 3492361]),
 (3640861, [12, 3640861]),
 (3856841, [12, 3856841]),
 (4276531, [12, 4276531]),
 (4566851, [12, 4566851]),
 (4952751, [12, 4952751]),
 (5398092, [12, 5398092]),
 (5489892, [12, 5489892]),
 (5546832, [12, 5546832]),
 (5660922, [12, 5660922]),
 (5732472, [12, 5732472]),
 (5741242, [12, 5741242]),
 (5748742, [12, 5748742]),
 (5874452,

In [98]:
%%bash
grep "422" /data/twitter/twitter_sample_small.txt

12	14222350
12	16234224
12	21042298
12	21422349
12	22422962
12	23422392
12	24227096
12	24534422
12	25371422
12	26323422
12	27422140
12	28994222
12	29422674
12	34226476
12	34422023
12	35422201
12	36764226
12	40067422
12	40247422
12	40656422
12	42203572
12	42214163
12	42228727
12	42247184
12	42248056
12	42248160
12	42250344
12	42252090
12	42253574
12	42263546
12	42263652
12	42272374
12	42273667
12	42277863
12	42284620
12	42286613
12	42299341
12	43378422
12	43972422
12	44221217
12	44224878
12	44225946
12	46342252
12	46442247
12	47680422
12	51422939
12	52142279
12	54224794
12	54227558
12	54587422
12	54742296
12	55000422
12	55042253
12	55246422
12	55394228
12	55422708
12	57227422
12	57942229
12	58144221
12	58422278
12	58942228
12	59558422
13	13300422
13	13565422
13	14226696
13	14422312
13	15422092
13	18422231
13	20422739
13	20759422
13	21024220
13	21422773
13	21822422
13	21942255
13	21992422
13	23382422
13	23422509
13	24220756
13	24342247
13	26422917
13	29142219
13	29242265
13	32242255
13	3